## Notebook: Contextual Anomaly Detection (CAD) Training

This is the first of a series of three notebooks that show how the Contextual Anomaly Detection (CAD) Accelerator can be used to train and deploy a prediction interval model into Monitor using the API service endpoints.
1.   Training: cookbooks/contextual_anomaly_train.ipynb
2.  Monitor Device Creation: cookbooks/contextual_anomaly_create_device.ipynb
3.   Model Deployment: cookbooks/contextual_anomaly_deploy_model.ipynb

### CAD Description

The CAD training job produces three prediction interval models that capture the normal operation (non-anomalous) behaviour of a given target variable based on a set of input features. 

These are point estimate multivariate regression machine learning models (denoted as base regressor) enhanced with conformal prediction statistical wrappers to produce a lower and upper bound that contain the target variable with probability 95% under normal operation (non-anomalous) conditions. Therefore, the probability of observing a target variable outside of the provided interval is 5% under normal operation conditions.


The three CAD models are saved in ONNX format to be compatible with MAS, the user can choose which model to deploy based on the performance summary provided by the training job.

### Wind Turbine Dataset Description: 

In this notebook we use the CAD Accelerator to learn a prediction interval model that covers the normal operation behaviour of the Average Reactive Power of a Wind turbine asset with probability 95%. We consider 4 input features to predict the target variable. These are Average Active Power, Average Generator Bearing 1 Temperature, Average Generator Bearing 2 Temperature, Average Wind Speed.







<a id='notebook_workflow'></a>
### Notebook Workflow
- [Imports](#imports)
- [Load KPI specification file (yaml)](#load_kpiyaml)
- [Load API Service config file (yaml)](#load_mfyaml)
- [Load Dataset](#load_dataset)
- [Generate Training Payload](#cad_payload)
- [Post Training Job](#cad_post)
- [Request Training Summary](#cad_summary)
- [Save Model Information](#cad_modelinfo)


<a id='imports'></a>
### Imports

In [ ]:
import pandas as pd
import requests
import time
from time import sleep
import yaml

!pwd

<a id='load_kpiyaml'></a>
### Load KPI specification file (yaml)



This file is a dictionary containing:

    - asset_id_column: column corresponding to asset id
    - data_name: train dataset file name
    - device_description: device description (optional)
    - mas_device_name: monitor device name (needed for device creation and/or deployment)
    - feature_columns: feature columns name as in first row of dataset csv file separated by ',' (e.g., P_avg,Rs_avg,Gb1t_avg,Ws_avg)
    - feature_names: feature columns interpretable names separated by ',' (same order as in feature_columns)
    - target_columns: target column name as in first row of dataset
    - target_names: target column interpretable name
    - timestamp_column: time stamp column name as in first row of dataset
    - timestamp_format: '%m/%d/%Y %H:%M'
    - inference_data_name: test dataset file name (optional for inference)
    - feature_map: dictionary mapping feature columns to descriptions (optional)

In [ ]:
input_file_name = "../config/Pavg_kpi.yml"

with open(input_file_name, 'r') as file:
    input_data = yaml.safe_load(file)

print('KPI specification file: ')
print(input_data)

<a id='load_mfyaml'></a>
### Load API Service config file (yaml)


This file is a dictionary containing:

    - endpoint_url: <ACTION: Replace with API Service endpoint URL>
    - train_recipe_endpoint: recipe/supervised-anomaly (DON'T CHANGE)
    - deploy_recipe_endpoint: deployment/monitor/model/create (DON'T CHANGE)
    - create_device_recipe_endpoint: deployment/monitor/device/create (DON'T CHANGE)

In [ ]:
model_factory_config_file_name = "../config/model_factory_config.yml"

with open(model_factory_config_file_name, 'r') as file:
    model_factory_config = yaml.safe_load(file)

print(model_factory_config)

<a id='load_dataset'></a>
### Load Dataset


In [ ]:
prefix = "../data/"
data_df_1 = pd.read_csv(prefix + input_data['data_name'])
data_df_1.head(10)


<a id='cad_payload'></a>
### Generate Training Payload



Training Payload Required Information (this will be read from the input_data dictionary loaded above)

- Payload:
    - "feature_columns": feature column names separated by ','
    - "target_columns": target column name separated
    - "feature_names": features descriptive names separated by ','
    - "target_names": target descriptive name
- files:
    - "data_file": .csv data file with column names matching "feature_columns" and "target_columns" as specified in payroll.
    

In [ ]:
payload = {'feature_columns': input_data['feature_columns'],
'feature_names': input_data['feature_names'],
'target_columns': input_data['target_columns'],
'target_names': input_data['target_names']}

data_folder = "../data/"

files=[
  ('data_file',(input_data['data_name'],open(data_folder + input_data['data_name'],'rb'),'text/csv'))
]

print('Payload :')
print(payload)

<a id='cad_post'></a>
### Post Training Job

In [ ]:
endpoint_url = model_factory_config["endpoint_url"]
training_endpoint_url = endpoint_url + model_factory_config["train_recipe_endpoint"]

headers = {
  'accept': 'application/json'
}
start_time = time.time()

response = requests.request("POST", training_endpoint_url, headers=headers, data=payload, files=files)

In [ ]:
post_r_json = response.json()
print(post_r_json)
job_id = post_r_json['job_id']
print()
print('job id:',job_id )

<a id='cad_summary'></a>
### Request Training Summary

The following code will obtain a summary after the training job is completed. This summary contains the performance of the three trained prediction interval models on test, train and validation data.
We report the following metrics:
- R2 score: indicates the R2 score of the base regressor, higher is better.
- RMSE:  indicates the root mean square error of the base regressor, lower is better.
- Coverage: indicates the probability of finding the groud truth target inside the produced interval (this value should be larger or equal to 0.95)
- Mean Width: Indicates the average prediction interval width
- Median Width: Indicates the median prediction interval width

In [ ]:

log_url = endpoint_url + "log/"
summary_url = endpoint_url + "summary/"

print('Summary URL :: ')
print(summary_url + job_id)
print()
while True:
    get_response = requests.get(summary_url + job_id, headers={})
    json_data = get_response.json()
    print("json_data",json_data)
    if 'status' in json_data:
        print('status :', json_data['status'])
        print()
        if json_data['status'] == 'DONE':
            print()
            print('-------------  Trained Models -------------  ')
            print()
            performance_dictionary = json_data['summary']['performance_dictionary']
            for model in performance_dictionary.keys():
                print('model name : ',model)
                for key in performance_dictionary[model]:
                    if key != 'performance':
                        print(key, ' : ',performance_dictionary[model][key])
                print()
                print('Performance : ')
                for dataset in performance_dictionary[model]['performance'].keys():
                    print(dataset + ' set')
                    print(performance_dictionary[model]['performance'][dataset])
                    print()


                print('----------------')
                print()
            finish_time = time.time()
            break

        else:
            print('STATUS : ',json_data['status'] )
            print('retry later .... ')
            sleep(2)


In [ ]:
print('Trained Model Names')
print(performance_dictionary.keys())
print('-----------------')
print()

''' !!ACTION :: Here you can choose any of the models in performance_dictionary.keys()'''
model_selected = list(performance_dictionary.keys())[0]

print('Selected Model is : ', model_selected)
for key in performance_dictionary[model_selected]:
    if key != 'performance':
        print(key, ' : ',performance_dictionary[model_selected][key])
print()
print('Performance : ')
for dataset in performance_dictionary[model_selected]['performance'].keys():
    print(dataset + ' set')
    print(performance_dictionary[model_selected]['performance'][dataset])
    print()
print('----------------')
print()

<a id='cad_modelinfo'></a>
### Save Model Information

The information of the selected model is saved in 'config/model_info.yml'. This information will be needed for model deployment into Monitor

In [ ]:
if 'status' in json_data:
    if json_data['status'] == 'DONE':
        output_data = {
            "onnx_model_uri" : performance_dictionary[model]['model_uri'],
            "train_job_id" : job_id,
            "mas_device_name" : input_data['mas_device_name']
        }
        with open("../config/model_info.yml","w") as file:
            yaml.dump(output_data, file)
        print('Model Information: ')
        print(output_data)